In [1]:
%load_ext autoreload
import os
os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [2]:
%autoreload 2
from src.database import ElasticTransformers

In [3]:
from sentence_transformers import SentenceTransformer

bert_embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [7]:
def embed_wrapper(ls):
    """
    Helper function which simplifies the embedding call and helps lading data into elastic easier
    """
    results=bert_embedder.encode(ls)
    results = [r for r in results]
    return results


In [8]:
et=ElasticTransformers(index_name='et-test')
et.ping()

2020-08-31 17:11:21,143 - src.logger - DEBUG - ping:32 - Ping successful
DEBUG:src.logger:Ping successful


True

# Size matters
Below we search for `security thead` in the tiny (1k records) vs the large (1.1mn records, 1100x) index. 
 - keyword matching takes 12ms on tiny and 45ms (4x) in the large index 
 - dense (embedding) search takse 80ms on tiny and 8400ms (105x) on the large index

In [9]:
%%time
_ = et.search('security thread','headline_text',index_name='et-tiny',type='dense',embedder=embed_wrapper, size=100)

2020-08-31 17:11:22,597 - src.logger - DEBUG - search:251 - Search DENSE security thread in et-tiny.headline_text returned 100 results of 100 requested
DEBUG:src.logger:Search DENSE security thread in et-tiny.headline_text returned 100 results of 100 requested


CPU times: user 688 ms, sys: 19.7 ms, total: 708 ms
Wall time: 317 ms


In [ ]:
%%time
_ = et.search('security thread','headline_text',index_name='et-tiny',type='match',embedder=embed_wrapper, size=100)

In [ ]:
%%time
_ = et.search('security thread','headline_text',type='match',embedder=embed_wrapper, size=100)


In [ ]:
%%time
_ = et.search('security thread','headline_text',type='dense',embedder=embed_wrapper, size=100)


In [ ]:
%%time
_ = et.search('news','headline_text',type='dense',embedder=embed_wrapper, size=5)


In [ ]:
%%time
_ = et.search('virus thread','headline_text',type='match',embedder=embed_wrapper, size = 100)

In [ ]:
import pandas as pd
df=pd.read_csv('data/abcnews-date-text.csv')

In [ ]:
df.shape